#Data 불러오기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#importing important libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as dates
from datetime import datetime
from statsmodels.tsa.arima.model import ARIMA

In [4]:
df=pd.read_csv('/content/중년_격렬.csv')
df = df.copy()
df.head()

,update_time,heart_rate1,event1,heart_rate2,event2,heart_rate3,event3,heart_rate4,event4,heart_rate5,...,heart_rate996,event996,heart_rate997,event997,heart_rate998,event998,heart_rate999,event999,heart_rate1000,event1000
0,2023-10-01 00:00:00,98,0,168,1,110,0,110,0,154,...,124,0,123,0,81,0,85,0,126,0
1,2023-10-01 00:10:00,91,0,93,0,162,0,146,0,153,...,142,0,122,0,135,0,138,0,134,0
2,2023-10-01 00:20:00,145,0,140,0,165,1,130,0,119,...,134,0,137,0,140,0,113,0,134,0
3,2023-10-01 00:30:00,127,0,104,0,132,0,119,0,112,...,130,0,108,0,123,0,82,0,142,0
4,2023-10-01 00:40:00,131,0,106,0,88,0,124,0,171,...,97,0,128,0,120,0,130,0,102,0


#반복문

In [5]:
df_copy = df.copy()
# 데이터를 df_train과 df_test로 분할
df_train = df_copy[:1008]
df_test = df_copy[1008:]


In [6]:
for i in range(1, 1001):
    # ARIMA 모델 학습
    model = ARIMA(df_train[f'heart_rate{i}'], order=(2,0,1))
    result = model.fit()

    start_idx = 1008  # 테스트 데이터셋의 시작 인덱스
    end_idx = 1008 + len(df_test)  # 테스트 데이터셋의 끝 인덱스

    forecast = result.get_prediction(start=start_idx, end=end_idx, dynamic=False)

    # 신뢰구간과 예측값 할당
    forecast_ci = forecast.conf_int()

  # 데이터프레임에 신뢰구간 추가
    df_test[f'lower_{i}'] = forecast_ci.iloc[:, 0]
    df_test[f'upper_{i}'] = forecast_ci.iloc[:, 1]

    # 'pred_y{i}' 열이 이미 존재하는 경우 삭제하고 다시 추가 (insert 메서드 사용)
    if f'pred_y{i}' in df_test.columns:
        df_test.drop(columns=[f'pred_y{i}'], inplace=True)

    # 'pred_y{i}' 열을 'event{i}' 뒤에 생성
    df_test.insert(df_test.columns.get_loc(f'event{i}') + 1, f'pred_y{i}', np.where((df_test[f'heart_rate{i}'] < df_test[f'lower_{i}']) | (df_test[f'heart_rate{i}'] > df_test[f'upper_{i}']), 1, 0))

    # 'pred{i}', 'lower_{i}', 'upper_{i} 삭제
    if f'pred{i}' in df_test.columns:
        df_test.drop(columns=[f'pred{i}'], inplace = True)
    if f'lower_{i}' in df_test.columns:
        df_test.drop(columns=[f'lower_{i}'], inplace = True)
    if f'upper_{i}' in df_test.columns:
        df_test.drop(columns=[f'upper_{i}'], inplace = True)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test[f'upper_{i}'] = forecast_ci.iloc[:, 1]
<ipython-input-6-9d01e8dececb>:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_test.insert(df_test.columns.get_loc(f'event{i}') + 1, f'pred_y{i}', np.where((df_test[f'heart_rate{i}'] < df_test[f'lower_{i}']) | (df_test[f'heart_rate{i}'] > df_test[f'upper_{i}']), 1, 0))
<ipython-input-6-9d01e8dececb>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u

In [7]:
df_test

,update_time,heart_rate1,event1,pred_y1,heart_rate2,event2,pred_y2,heart_rate3,event3,pred_y3,...,pred_y997,heart_rate998,event998,pred_y998,heart_rate999,event999,pred_y999,heart_rate1000,event1000,pred_y1000
1008,2023-10-08 00:00:00,100,0,0,132,0,0,108,0,0,...,0,119,0,0,107,0,0,102,0,0
1009,2023-10-08 00:10:00,126,0,0,123,0,0,96,0,0,...,0,113,0,0,143,0,0,138,0,0
1010,2023-10-08 00:20:00,141,0,0,74,0,0,95,0,0,...,0,77,0,0,84,0,0,103,0,0
1011,2023-10-08 00:30:00,119,0,0,84,0,0,98,0,0,...,0,108,0,0,130,0,0,108,0,0
1012,2023-10-08 00:40:00,156,0,0,95,0,0,101,0,0,...,0,133,0,0,134,0,0,106,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1345,2023-10-10 08:10:00,124,0,0,153,0,0,112,0,0,...,1,125,0,0,133,0,0,136,0,0
1346,2023-10-10 08:20:00,103,0,0,115,0,0,135,0,0,...,0,143,0,0,106,0,0,95,0,0
1347,2023-10-10 08:30:00,124,0,0,138,0,0,165,1,1,...,0,84,0,0,139,0,0,127,0,0
1348,2023-10-10 08:40:00,188,1,1,121,0,0,122,0,0,...,0,97,0,0,94,0,0,83,0,0


In [8]:
def performance(TP, TN, FP, FN):

    N = TP+TN+FP+FN

    Sensitivity=(TP/(TP+FN))*100

    Specificity=(1-FP/(TN+FP))*100

    Accuracy=((TP+TN)/(TP+TN+FP+FN))*100

    PPV= (TP/(TP+FP))*100

    NPV= (TN/(TN+FN))*100

    result = {
        'Sensitivity': [Sensitivity],
        'Specificity': [Specificity],
        'Accuracy':[Accuracy],
        'PPV':[PPV],
        'NPV':[NPV]
    }

    result_2 = pd.DataFrame(result)

    return result_2

In [9]:
result = pd.DataFrame()

for i in range(1,1001):

    TP = len(df_test[(df_test[f'event{i}']==1)&(df_test[f'pred_y{i}']==1)])
    TN = len(df_test[(df_test[f'event{i}']==0)&(df_test[f'pred_y{i}']==0)])
    FN = len(df_test[(df_test[f'event{i}']==1)&(df_test[f'pred_y{i}']==0)])
    FP = len(df_test[(df_test[f'event{i}']==0)&(df_test[f'pred_y{i}']==1)])


    result_i = performance(TP=TP, TN=TN, FP=FP, FN=FN)

    result = pd.concat([result, result_i])


result.head(2)

,Sensitivity,Specificity,Accuracy,PPV,NPV
0,100.0,98.170732,98.245614,70.0,100.0
0,100.0,99.076923,99.122807,85.0,100.0


In [11]:
result_201_중년_격렬_5 = pd.DataFrame(index=['mean'])

result_201_중년_격렬_5['mean_sens'] = result['Sensitivity'].mean()
result_201_중년_격렬_5['std_sens'] = result['Sensitivity'].std()
result_201_중년_격렬_5['mean_spec'] = result['Specificity'].mean()
result_201_중년_격렬_5['std_spec'] = result['Specificity'].std()
result_201_중년_격렬_5['mean_accu'] = result['Accuracy'].mean()
result_201_중년_격렬_5['std_accu'] = result['Accuracy'].std()
result_201_중년_격렬_5['mean_ppv'] = result['PPV'].mean()
result_201_중년_격렬_5['std_ppv'] = result['PPV'].std()
result_201_중년_격렬_5['mean_npv'] = result['NPV'].mean()
result_201_중년_격렬_5['std_npv'] = result['NPV'].std()

result_201_중년_격렬_5

,mean_sens,std_sens,mean_spec,std_spec,mean_accu,std_accu,mean_ppv,std_ppv,mean_npv,std_npv
mean,98.990184,4.101998,99.203643,0.692037,99.197661,0.648661,85.709306,11.252804,99.957731,0.164483


In [12]:
from google.colab import files
result_201_중년_격렬_5.to_csv('result_201_중년_격렬_5.csv', index=False)


# CSV 파일 다운로드
files.download('result_201_중년_격렬_5.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>